In [ ]:
'''
This cluster of functions allows a user to iterate through a designated field within a feature 
class, select all features for each unique attribute of that designated field, and export those
features as their own feature class to a designated geodatabase. 
'''

In [ ]:
'''
Imports 
'''
import os 

'''
Parameters
'''
# The feature class to be iterated through 
input_path      = r"https://arcgis.metc.state.mn.us/server/rest/services/GISLibrary/MetroCollaborativeTrailsBikeways/FeatureServer/0"
input_path      = r"https://arcgis.metc.state.mn.us/server/rest/services/GISLibrary/Bikeways/FeatureServer/0"
input_path      = r"https://arcgis.metc.state.mn.us/server/rest/services/GISLibrary/RegionalBikewaysInventory/FeatureServer/0"
input_name      = 'Bikeways'

# The designated field 
attribute_field = 'RegType'

# Selection type ('Keywords', 'Automatic', 'Manual')
selection_type  = 'Automatic'

# Attribute groups 
keywords        = []

# Rename feature classes 
fc_rename = True

# Filter 
filter_clause = "RegStat = 1"

# The output geodatabase 
clear_gdb       = True 
gdb             = r"C:\DALE\Personal\Minnesota\MyProject\Bikeways.gdb"

In [ ]:
'''
Clear geodatabase
'''

def clear_existing_gdb(gdb): 
    
    arcpy.env.workspace = gdb 

    for fc in arcpy.ListFeatureClasses(): 
        fc_path = os.path.join(gdb, fc)
        arcpy.management.Delete(fc_path)
        print(fc_path)
        
    return gdb 

In [ ]:
'''
Feature Class Rename Dictionary 
'''

fc_rename_dict    = {}
fc_rename_dict[0] = 'Unknown'
fc_rename_dict[1] = 'Signage'
fc_rename_dict[2] = 'Bikeable_Shoulders'
fc_rename_dict[3] = 'Painted_Bike_Lanes'
fc_rename_dict[4] = 'Below_Curb_Protected'
fc_rename_dict[5] = 'Above_Curb_Protected'
fc_rename_dict[6] = 'Street_Adjacent_Trail'
fc_rename_dict[7] = 'Independent_Trail'

In [ ]:
'''
This organizes the dictionary by a list of keywords
'''

def create_selection_dictionary_by_keyword(input_path, attribute_field, keywords):
    
    # Create selection dictionary
    selection_dictionary = {} 
    
    for key in keywords: 
        selection_dictionary[key] = []

    # Iterate through feature class to populate dictionary
    keyed_attributes = [] 
    
    with arcpy.da.SearchCursor(input_path, [attribute_field]) as cursor: 
        for row in cursor: 
            if row[0] in keyed_attributes: 
                pass 
            else: 
                for key in keywords: 
                    if key in row[0]: 
                        selection_dictionary[key].append(row[0])
                        keyed_attributes.append(row[0])
    
    return selection_dictionary

In [ ]:
'''
This filters a feature class to get rid of unwatned features. 
The filter clause designates features that WILL be kept 
'''

def filter_feature_class(input_path, gdb, input_name, filter_clause): 
    
    # Designate output path 
    filtered_input      = os.path.join(gdb, input_name)
    
    # Export feature class 
    arcpy.conversion.ExportFeatures(input_path, filtered_input, filter_clause)
    
    return filtered_input 

In [ ]:
'''
This creates a dictionary of feature class names (attribute) and field values.
The field values will be used to populate SQL clauses.
'''

def create_selection_dictionary(filtered_input, attribute_field): 
    
    # Create selection dictionary
    selection_dictionary = {} 

    # Iterate through feature class to populate dictionary
    with arcpy.da.SearchCursor(filtered_input, [attribute_field]) as cursor: 
        for row in cursor: 

            # Identify attribute 
            attribute      = row[0]
            attribute_name = row[0]
            
            try: 
                attribute_name = attribute.replace(" ", "_")
                attribute_name = attribute_name.replace("-", "_")
                attribute_name = attribute_name.replace(".", "_")
                attribute_name = attribute_name.replace(",", "_")
                attribute_name = attribute_name.replace("'", "")
                attribute_name = attribute_name.replace("/", "_")
                attribute_name = attribute_name.replace(">=", "Greater")
                attribute_name = attribute_name.replace(">", "Greater")
                attribute_name = attribute_name.replace("<=", "Less")
                attribute_name = attribute_name.replace("<", "Less")
                attribute_name = attribute_name.replace("5", "FiveFt")
            except: 
                pass

            # Populate dictionary 
            selection_dictionary[attribute_name] = attribute
        
    return selection_dictionary

'''
Export through the dictionary 
'''

def export_from_selection_dictionary(selection_dictionary, gdb, fc_rename): 
    
    # Iterate through selection dictionary 
    for attribute_name, attribute in selection_dictionary.items(): 

        # Assemble SQL clause 
        if selection_type == 'Keyword': 
            
            sql = ""

            if len(attributes) == 1: 
                sql = f"{attribute_field} = '{attribute}'"
            else: 
                for attribute in attributes:
                    if sql == "": 
                        sql = f"{attribute_field} = '{attribute}'"
                    else: 
                        sql = sql + ' Or ' + f"{attribute_field} = '{attribute}'"
                        
        else: 
            if type(attribute_name) != int: 
                sql = f"{attribute_field} = '{attribute_name}'"
            else: 
                sql = f"{attribute_field} = {attribute_name}"
        
        # Define output path 
        if fc_rename == True: 
            attribute_name = fc_rename_dict[attribute_name]
            
        output_path = os.path.join(gdb, attribute_name)
        
        try: 
            # Export 
            arcpy.conversion.ExportFeatures(filtered_input, output_path, sql) 
            print(f'SQL:           {sql}')
            print(f'Feature class: {attribute_name}')
            print(f'Exported:      {output_path}')
            print('---')
        except Exception as e:
            print(e)
            print(sql)
            print(output_path)
    
    return gdb 

'''
Combine both functions 
'''

def export_features_by_attribute(filtered_input, attribute_field, gdb, selection_type, keywords): 
        
    # Create selection dictionary
    if selection_type == 'Automatic': 
        print('Automatic Selection')
        print('-------------------')
        selection_dictionary = create_selection_dictionary(filtered_input, attribute_field)
        
    elif selection_type == 'Keywords': 
        print('Keyword Selection')
        print('-------------------')
        selection_dictionary = create_selection_dictionary_by_keyword(filtered_input, attribute_field, keywords)
        
    # Export from selection dictionary 
    gdb = export_from_selection_dictionary(selection_dictionary, gdb, fc_rename)
    
    return selection_dictionary, gdb 

In [ ]:
"""
Run Function 
"""
if clear_gdb == True: 
    gdb = clear_existing_gdb(gdb)
    
filtered_input            = filter_feature_class(input_path, gdb, input_name, filter_clause)
selection_dictionary, gdb = export_features_by_attribute(filtered_input, attribute_field, gdb, selection_type, keywords)